# Import National Monuments Data

I copy/pasted the table from [Wikipedia's List of National Monuments](https://en.wikipedia.org/w/index.php?title=List_of_National_Monuments_of_the_United_States&oldid=757281136), but that doesn't contain the area of each monument.  So I'll scrape it.

In [1]:
import pandas as pd
import numpy as np
import pickle

pd.set_option('display.max_rows', 500)

In [2]:
nm = pd.read_csv('national_monuments_wikipedia.csv')

In [3]:
nm.columns

Index([u'National Monument Name', u'Photo', u'Agency', u'Location',
       u'Date Established', u'Description'],
      dtype='object')

In [4]:
nm['National Monument Name'] = nm['National Monument Name'].str.decode('utf-8').str.strip()

In [5]:
nm['established'] = pd.to_datetime(nm['Date Established'])

In [6]:
nm['year'] = nm['established'].apply(lambda x: x.year)

## Load Scraped National Monuments

In [7]:
with open('national_monuments_areas.pkl', 'r') as ifile:
    dd = pickle.load(ifile)
print len(dd)

126


Need to do two things:
1. Convert to sq km
2. merge with nm data frame
1. TEXT PARSING SUCKS

In [8]:
def acres_to_sqkm(a):
    return float(a.replace(',', '')) * 0.00404686

def sqmi_to_sqkm(a):
    return float(a.replace(',', '')) * 2.58999

In [9]:
data = [[], []]
for k in sorted(dd):
    data[0].append(k.decode('utf-8').strip())
    data[1].append(dd[k])
nma = pd.DataFrame(np.c_[data[0], data[1]], columns=['National Monument Name', 'Area'])

In [10]:
data2 = [[], []]
for row in nma.iterrows():
    a = row[1]['Area'].split()
    if row[1]['National Monument Name'] == '':
        print row
    if len(a) >= 2:
        if a[1].lower() != 'acres':
            if a[1].lower() == 'square':
                data2[0].append(row[1]['National Monument Name'])
                data2[1].append(sqmi_to_sqkm(a[0]))
            else:
                data2[0].append(row[1]['National Monument Name'])
                data2[1].append(0.)
        else:
            data2[0].append(row[1]['National Monument Name'])
            data2[1].append(acres_to_sqkm(a[0]))
    else:
        data2[0].append(row[1]['National Monument Name'])
        data2[1].append(0.)

In [12]:
nmb = pd.DataFrame(np.c_[data2[0], data2[1]], columns=['National Monument Name', 'sqkm'])

In [14]:
nm = nm.merge(nmb, how='outer', on='National Monument Name')
len(nm)

126